In [1]:
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:   
    sys.exit("please declare environment variable 'SUMO_HOME'")
    
import sumolib

In [7]:
import pickle

city = "lac"

with open('data/' + city + '_features.pkl', 'rb') as f:
    features = pickle.load(f)

with open('data/' + city + '_speeds.pkl', 'rb') as f:
    speeds = pickle.load(f)
    
with open('data/small_coords.pkl', 'rb') as f:
    coords = pickle.load(f) 

In [9]:
import pandas as pd

edges = pd.read_csv('data/' + city + '.csv', names=["source", "dest"])
edges.head()

,source,dest
0,4020268087,1720047709
1,317663450,310688433
2,317663450,317663724
3,1834922437,1834922447
4,305273050,305273051


In [10]:
# Create node XML
import xml.etree.cElementTree as ET

nodes = ET.Element("nodes")
for n in set(edges["source"]).union(set(edges["dest"])):
    v = str(n)
    long, lat = coords[v]
    node = ET.SubElement(nodes, "node", id=v, x=str(long), y=str(lat))
    
tree = ET.ElementTree(nodes)
tree.write("out/" + city + ".nodes.xml")

In [11]:
# Create edge XML

edgeXML = ET.Element("edges")
i = 0
for i, e in enumerate(edges.itertuples()):
    feat = features[tuple([str(v) for v in e[1:3]])]
    numLanes = str(int(feat[2]))
    ET.SubElement(edgeXML, "edge", {"id": str(i), "from": str(e[1]), "to": str(e[2]), 
                                    "numLanes": str(int(feat[2])), "speed": str(feat[3] * 0.44704), "length": str(feat[4])})
    
tree = ET.ElementTree(edgeXML)
tree.write("out/" + city + ".edges.xml")

In [12]:
import xml.etree.cElementTree as ET
import numpy as np

net = sumolib.net.readNet('out/' + city + '.net.xml')

with open('data/' + city + '_flows.pkl', 'rb') as f:
    flows = pickle.load(f)
    
data = ET.Element("data")

for t in range(1):
    interval = ET.SubElement(data, "interval", id=str(t), begin=str(3600 * t), end=str(3600 * (t + 1)))
    
    for i, e in enumerate(net.getEdges()):
        key = (str(e.getFromNode().getID()), str(e.getToNode().getID()))
        if key in flows.keys():
            flow = str(int(flows[key]))
            try:
                speed = speeds[key] * 0.44704 # Convert from mph to m/s
                ET.SubElement(interval, "edge", id=str(e.getID()), entered=flow, speed=str(speed))
            except KeyError:
                ET.SubElement(interval, "edge", id=str(e.getID()), entered=flow)
            
            
tree = ET.ElementTree(data)
tree.write("out/routeSampler/" + city + ".edgedata.xml")

ValueError: unknown url type: 'out/lac.net.xml'